In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [ ]:
baseball = pd.read_csv('best_model/engineered_data.csv')

In [ ]:
X = baseball.drop(['Salary'], axis = 1)
y = baseball['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 621)

In [ ]:
cat_columns = ['Tm', 'Lg', 'Acquired', 'Bat']
num_columns = [col for col in X.columns if col not in cat_columns + ['Pos_C', 'Pos_1B', 'Pos_2B', 'Pos_3B', 'Pos_SS', 'Pos_OF']]

cat_transformer = Pipeline(
    steps = [
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

num_transformer = Pipeline(
    steps = [
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, num_columns),
        ('cat', cat_transformer, cat_columns)
    ], remainder = 'passthrough'
)

X_transform = preprocessor.fit_transform(X)

selected_features = np.concatenate([
    np.array(num_columns),
    np.array(preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(cat_columns)),
    np.array(['Pos_C', 'Pos_1B', 'Pos_2B', 'Pos_3B', 'Pos_SS', 'Pos_OF'])
])


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dropout, Dense
from bayes_opt import BayesianOptimization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import warnings
import joblib

warnings.filterwarnings('ignore')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size = .2, random_state = 621)

In [ ]:
def dnn_model_score(neurons, dropout_rate, learning_rate, epochs, batch_size, patience, num_layers, **layer_neurons):
    model = Sequential()
    model.add(Dense(int(neurons), activation='relu', input_shape = (X_train.shape[1],)))
    model.add(Dropout(dropout_rate))

    for i in range(1, int(num_layers) + 1):
        model.add(Dense(int(layer_neurons[f'layer_neurons_{i}']), activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation = 'linear'))

    optimizer = Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_squared_error', optimizer = optimizer)

    es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

    model.fit(X_train, y_train, validation_split = .2, epochs = int(epochs), batch_size = int(batch_size), callbacks = es, verbose = 0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return -mse

pbounds = {'neurons': (32, 256),
           'dropout_rate': (0.0, 0.5),
           'learning_rate': (0.01, 1),
           'epochs' : (100, 500),
           'batch_size' : (32, 500),
           'patience' : (20, 50),
           'num_layers': (1, 5)}

for i in range(1, 6):
    pbounds[f'layer_neurons_{i}'] = (32, 256)

optimizer = BayesianOptimization(f = dnn_model_score, pbounds = pbounds, random_state = 42)

optimizer.maximize(init_points = 5, n_iter = 10)

best_params = optimizer.max['params']
print("Best Hyperparameters:", best_params)


In [ ]:
batch_size = int(best_params['batch_size'])
dropout_rate = best_params['dropout_rate']
epochs = int(best_params['epochs'])
neurons = []
neurons.append(int(best_params['neurons']))
neurons.append(int(best_params['layer_neurons_1']))
neurons.append(int(best_params['layer_neurons_2']))
neurons.append(int(best_params['layer_neurons_3']))
neurons.append(int(best_params['layer_neurons_4']))
neurons.append(int(best_params['layer_neurons_5']))
learning_rate = best_params['learning_rate']
num_layers = int(best_params['num_layers'])
patience = int(best_params['patience'])

In [ ]:
model = Sequential()

model.add(Dense(name = 'Dense1', units = neurons[0], input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dropout(name = 'Dropout1', rate = dropout_rate))

for i in range(1, num_layers + 1):
    model.add(Dense(name = f'Dense{i + 1}', units = neurons[i], activation = 'relu'))
    model.add(Dropout(name = f'Dropout{i + 1}', rate = dropout_rate))

model.add(Dense(name = 'Output', units = 1, activation = 'linear'))

optimizer = Adam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, loss = 'mean_squared_error')

model.summary()

In [ ]:
es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

history = model.fit(X_train, y_train, validation_split = .2, batch_size = batch_size, epochs = epochs, callbacks = es, verbose = 2)

In [ ]:
test_preds = model.predict(X_test)
train_preds = model.predict(X_train)

In [ ]:
train_rmse = mean_squared_error(y_train, train_preds)
test_rmse = mean_squared_error(y_test, test_preds)

print('Deep Neural Network Metrics:')
print(f'Train RMSE: {np.sqrt(train_rmse)}')
print(f'Test RMSE: {np.sqrt(test_rmse)}')

In [ ]:
# save the model, pca, and preprocessor so that new data can be fit using the same criteria
#model.save('best_model/best_model.keras')
#joblib.dump(pca, 'dnn/pca45.joblib')
#joblib.dump(preprocessor, 'best_model/best_preprocssor.joblib')
